# MLAA Assessment 2 Experiment 5
## Ronik Jayakumar
## 24680264



## Context
In this experiemnt, we will be exploring undersampling techniques using Near Miss undersampler. The machine learning model chosen for the 5th and final experiment will be the Gradient Boosting Classifier.

### 0. Import Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import NearMiss
from sklearn.datasets import make_classification
from collections import Counter
from sklearn.metrics import classification_report, accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import learning_curve
from sklearn.ensemble import GradientBoostingClassifier

### 1.   Load Dataset

In [2]:
df = pd.read_csv('/Users/ronikj/Downloads/df.csv')
df.head()

,gender_no,car_segment,age_of_vehicle_years,sched_serv_warr,sched_serv_paid,total_paid_services,total_services,mth_since_last_serv,Target
0,1.0,0,9,2,3,5,6,9,0
1,0.0,2,6,10,10,9,10,6,0
2,1.0,1,9,10,10,10,10,7,0
3,1.0,1,5,8,8,5,6,4,0
4,0.0,2,8,9,10,9,8,5,0


### 2. Explore Dataset

In [3]:
df.isnull().sum()

gender_no               0
car_segment             0
age_of_vehicle_years    0
sched_serv_warr         0
sched_serv_paid         0
total_paid_services     0
total_services          0
mth_since_last_serv     0
Target                  0
dtype: int64

The dataset has been cleaned in experiment 1 and all the features will be kept constant for our project. The aim of our project is to explore into the best performing model. The features will continue to be kept constant due to the good results we have been acheiving.

### 3. Explore Target variable

In [4]:
target = df['Target']

In [5]:
target.value_counts()

0    127759
1      3520
Name: Target, dtype: int64

As we see a large imbalance within the dataset, the rows of the dataset will be processed to deal with the class imbalance that persists.

### 4. Explore Variables of Interest

The variables of interest have been kept constant from Experiment 1. This is due to the business finalising on the questions to be asked in experiment 1 and sticking to it to ensure questionaires do not get too lengthy. 

Therefore the variables of interest will be kept constant - gender_no, car_segment, age_of_vehicle_years, sched_serv_warr, sched_serv_paid, total_paid_services, total_services, mth_since_last_serv

### 5. Prepare Data

The data has been prepared in Experiment 1 and data preparation will be kept constant throughout all experiments to ensure uniformity for accurate model assessment.

### 6. Split Dataset

In [6]:
X = df.drop(['Target'], axis=1)
y = df['Target']

In [7]:
X_train_base, X_temp_base, y_train_base, y_temp_base = train_test_split(X, y, test_size=0.3, random_state=42)
X_test_base, X_val_base, y_test_base, y_val_base = train_test_split(X_temp_base, y_temp_base, test_size=0.5, random_state=42)

In [8]:
print("Baseline Training set shape:", X_train_base.shape, y_train_base.shape)
print("Baseline Validation set shape:", X_val_base.shape, y_val_base.shape)
print("Baseline Testing set shape:", X_test_base.shape, y_test_base.shape)

Baseline Training set shape: (91895, 8) (91895,)
Baseline Validation set shape: (19692, 8) (19692,)
Baseline Testing set shape: (19692, 8) (19692,)


### 7. Baseline Model

In [9]:
base_model = GradientBoostingClassifier(random_state=42)
base_model.fit(X_train_base, y_train_base)

GradientBoostingClassifier(random_state=42)

In [10]:
y_train_base_pred = base_model.predict(X_train_base)
y_val_base_pred = base_model.predict(X_val_base)

In [11]:
print("Training Classification Report:")
print(classification_report(y_train_base, y_train_base_pred))

print("Validation Classification Report:")
print(classification_report(y_val_base, y_val_base_pred))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     89427
           1       0.90      0.51      0.65      2468

    accuracy                           0.99     91895
   macro avg       0.94      0.75      0.82     91895
weighted avg       0.98      0.99      0.98     91895

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19181
           1       0.91      0.51      0.65       511

    accuracy                           0.99     19692
   macro avg       0.95      0.75      0.82     19692
weighted avg       0.99      0.99      0.98     19692



In [12]:
y_test_base_pred = base_model.predict(X_test_base)

In [13]:
print("Validation Classification Report:")
print(classification_report(y_test_base, y_test_base_pred))

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19151
           1       0.91      0.51      0.65       541

    accuracy                           0.99     19692
   macro avg       0.95      0.75      0.82     19692
weighted avg       0.98      0.99      0.98     19692



In the baseline model we notice the following details:
1. The majority class (0) gets predicted well in all the three classification reports.
2. Prediction scores for the minority class (1) requires imporvement as we see a sharp decline in the parametric scores.

### 8. Train Machine Learning Model

In [14]:
nearmiss_sampler = NearMiss(version=3)
X_undersampled, y_undersampled = nearmiss_sampler.fit_resample(X, y)

In [15]:
X_train, X_temp, y_train, y_temp = train_test_split(X_undersampled, y_undersampled,
                                                    test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

In [16]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

gb_clf = GradientBoostingClassifier()

grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [18]:
model = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=50, random_state=42)

In [19]:
model.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=5, n_estimators=50, random_state=42)

In [20]:
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

In [21]:
y_test_pred = model.predict(X_test)

### 9. Assess Model Performance

In [22]:
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Training Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.89      0.82      2437
           1       0.87      0.73      0.79      2491

    accuracy                           0.81      4928
   macro avg       0.82      0.81      0.81      4928
weighted avg       0.82      0.81      0.81      4928

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       329
           1       0.80      0.75      0.77       305

    accuracy                           0.79       634
   macro avg       0.79      0.79      0.79       634
weighted avg       0.79      0.79      0.79       634

Test Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.86      0.80       754
           1       0.83      0.70      0.76       724

    accuracy                           0.78      1478
   macro avg       0.79      0.78 

We see the follwing:
1. The gradient boosting classifier works well on the dataset considering past models such as logitic regression.
2. Undersampling imporves the model performance.
3. More methods need to be explored into as oversampling methods faired much higher accuracy and f1 scores as compared to the Near Miss undersampling method.